<a href="https://colab.research.google.com/github/wubinstu/roop/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/wubinstu/roop.git
workdir = "/content/roop"

In [ ]:
# pre roop

def version_to_number(version):
    parts = version.split('.')
    major = int(parts[0].replace('v', ''))
    minor = int(parts[1])
    patch = int(parts[2])
    return major, minor, patch


Version = "v1.1.0" #@param ["v1.0.1","v1.1.0","v1.2.1","v1.3.0","v1.3.1","v1.3.2"]
vdict = {
    "v1.0.1": "773ce17897cd6507d1ac17132ccc64ff6b6f26cb",
    "v1.1.0": "3d02b267663e4badc5c7936d5295effcae3dfbc0",
    "v1.2.1": "312208a41102ba86d2454ae8efc9d3f0b786a6e7",
    "v1.3.0": "142284916a5448acdf1a7fdca56d01c5aecafdad",
    "v1.3.1": "e0642702f388ea16089a1d4a929728fa69457d60",
    "v1.3.2": "c2d1feb17a9c51061b52cae5897136528f3b80cc",
}
commit=vdict.get(Version, "Invalid Version")
vcmd= f"checkout {commit}"
!git -C {workdir} $vcmd

import os
if os.path.exists(f"{workdir}/media") == False:
    !mkdir {workdir}/media
if os.path.exists(f"{workdir}/models") == False:
    !mkdir {workdir}/models

In [ ]:
# 安装依赖
%pip install -r {workdir}/requirements.txt

In [ ]:
# 下载模型
if os.path.exists(f"/content/GFPGANv1.4.pth") == False:
    !wget https://github.com/wubinstu/roop/releases/download/models/GFPGANv1.4.pth -P /content/
if os.path.exists(f"/content/inswapper_128.onnx") == False:
    !wget https://github.com/wubinstu/roop/releases/download/models/inswapper_128.onnx -P /content/


if os.path.exists(f"{workdir}/models/GFPGANv1.4.pth") == False:
    !cp /content/GFPGANv1.4.pth {workdir}/models/
if os.path.exists(f"{workdir}/models/inswapper_128.onnx") == False:
    !cp /content/inswapper_128.onnx {workdir}/models/

In [ ]:
# 使用通配符匹配文件, 以后缀区分, 注意不要让目录中出现同后缀文件
source = f"{workdir}/media/*.jpg"
target = f"{workdir}/media/*.mp4"
output = f"{workdir}/media/out.mkv"

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# cp file
!cp -rf /content/drive/MyDrive/? {workdir}/media/in.mp4
# !cp -rf /content/drive/MyDrive/? {workdir}/media/in.jpg

In [ ]:
# starting

Device = "cuda" #@param ["cuda", "cpu"]
Processor = "face_swapper" #@param ["face_swapper face_enhancer", "face_swapper","face_enhancer"]
VideoEncoder = "libx264" #@param ['libx264', 'libx265', 'libvpx-vp9', 'h264_nvenc', 'hevc_nvenc']
VideoQuality = "35" #@param {type:"string"}
TempFormat = "jpg" #@param ["jpg", "png"]
TempQuality = "0" #@param {type:"string"}
TempFormat = "--temp-frame-format " + TempFormat
TempQuality = "--temp-frame-quality " + TempQuality

KeepFPS = True #@param {type:"boolean"}
SkipAudio = False #@param {type:"boolean"}
KeepAudio = !SkipAudio
KeepFrames = False #@param {type:"boolean"}
ManyFaces = False #@param {type:"boolean"}


KeepFPS ="--keep-fps" if KeepFPS==True else ""
KeepFrames ="--keep-frames" if KeepFrames==True else ""
ManyFaces ="--many-faces" if ManyFaces==True else ""

# for 1.0.*,  1.1.*
KeepAudio ="--keep-audio" if KeepAudio==True else ""
# for 1.2.*,  1.3.*
SkipAudio ="--skip-audio" if SkipAudio==True else ""


# 将版本号转换成数字
Audio = ""
current_major, current_minor, current_patch = version_to_number(Version)
target_major, target_minor, target_patch = version_to_number("v1.2.1")
if (current_major, current_minor, current_patch) < (target_major, target_minor, target_patch):
    Audio = KeepAudio
    TempFormat = ""
    TempQuality = ""
    VideoEncoder = "--video-encoder " + VideoEncoder
    VideoQuality = "--video-quality " + VideoQuality
else:
    Audio = SkipAudio
    VideoEncoder = "--output-video-encoder " + VideoEncoder
    VideoQuality = "--output-video-quality " + VideoQuality


cmd = f"""{workdir}/run.py --execution-provider {Device} \
                        -s {source} -t {target} -o {output} \
                        --frame-processor {Processor} \
                        {VideoEncoder} {VideoQuality} \
                        {KeepFPS} {Audio} {KeepFrames} {ManyFaces} \
                        {TempFormat} {TempQuality}"""
print("cmd:"+cmd)

!python $cmd